# Các hướng dẫn cơ bản

Để sử dụng TensorFlow, chúng ta cần hiểu cách TensorFlow:

- Biểu diễn các phép tính dưới dạng graph
- Chạy graph trong Session
- Biễu diễn dữ liệu dưới dạng tensor 
- Lưu trạng thái với Variable 
- Sử dụng feeds và fetches để nhập và xuất dữ liệu ra khỏi các operation

TensorFlow là một hệ thống ngôn ngữ lập trình, trong đó ta biểu diễn các tính toán dưới dạng <b>graph</b>. Các nodes trong graph được gọi là các <b>operations</b>.

Một operation có thể nhận 0 hoặc nhiều hơn 0 Tensor, thực hiện một số tính toán, và tạo ra 0 hoặc nhiều hơn 0 Tensor.

<b>Tensor</b> là một typed multi-dimensional array. Ví dụ, một nhóm các hình ảnh có thể được biểu diễn bằng một 4-D array gồm các số thực. Bốn chiều của array này là [batch, height, width, channels], tương ứng với số hình ảnh, chiều cao của một ảnh, độ rộng của một ảnh, và số channel. Số channel của một ảnh tùy thuộc vào tính chất của ảnh, ví dụ ảnh đen trắng thì số channel là 1 do mỗi điểm ảnh chỉ cần 1 giá trị để thể hiện độ đen-trắng, còn ảnh màu RGB thì cần 3 channels để thể hiện độ đỏ-xanh lá-xanh dương tương ứng...

TensorFlow sử dụng graph để thể hiện toàn bộ quá trình tính toán. Để thực hiện tính toán trong graph, graph phải được chạy trong một <b>Session</b>. Một Session sẽ đưa các operation trong graph vào CPU hoặc GPU để thực hiện chúng.

Tutorials taken from "Hello, Tensorflow" by Aaron Schumacher, 2016 and "Getting started with TensorFlow" by Giancarlo Zaccone, 2016.

## 1. Xây dựng TensorFlow graph đơn giản nhất
Đầu tiên chúng ta import tensorflow và viết tắt nó là <i>tf</i>.

In [1]:
import tensorflow as tf

Dùng lệnh $get\_default\_graph()$ để thu được graph có sẵn.

In [2]:
graph = tf.get_default_graph()

Như đã đề cập, mỗi node trong graph là một operation. Hàm $get\_operations()$ sẽ trả về các operations có trong graph.

In [3]:
graph.get_operations()

[]

Lúc này graph chưa có operation nào cả, nhưng ta sẽ khởi tạo một hằng số với giá trị $1.0$.

In [4]:
input_value = tf.constant(1.0)

Hằng số này nằm trong một node, hay một operation trong graph.

In [5]:
operations = graph.get_operations()
operations

Từ output ta thấy được đây là một Operation được định nghĩa tại đây: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/framework/ops.py 

Ta có thể dùng $node\_def$ để thấy rõ hơn cấu trúc của operation này.

In [6]:
operations[0].node_def

name: "Const"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: 1.0
    }
  }
}

Chúng ta cũng có thể đánh giá "input_value" theo cách thông thường, nhưng sẽ chỉ thu được những thông tin sau.

In [6]:
input_value

<tf.Tensor 'Const:0' shape=() dtype=float32>

Bạn hãy thử tìm xem kết quả trên thu được nhờ hàm nào trong https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/framework/ops.py?  

<b>Trả lời:</b>

Kết quả cho thấy "input_value" là một hằng số, 32-bit float tensor không có chiều, nhưng kết quả không cho ta biết giá trị của hằng số này là gì. Để đánh giá "input_value" và nhận được giá trị số của nó, chúng ta cần đến một "session".

In [16]:
sess = tf.Session()
sess.run(input_value)

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

Lưu ý, ngoài $Session()$, chúng ta sẽ sử dụng $InteractiveSession()$ rất nhiều trong chương trình, vì với $InteractiveSession$, chúng ta không cần viết $sess.run(input\_value)$, mà dùng cách ngắn gọn hơn là $input\_value.eval()$.

In [17]:
sess = tf.InteractiveSession()
input_value.eval()

1.0

Hãy nhớ đóng session lại cuối chương trình!

In [15]:
sess.close()

## 2. Xây dựng TensorFlow neuron
Trong phần này, chúng ta sẽ mô phỏng lại một neuron đơn giản. Neuron này chỉ có một tham số cần tìm, đó là $weight$. Neuron sẽ nhận một giá trị input, và xuất ra một giá trị output theo công thức $$output = weight * input$$

Neuron sẽ bắt đầu với giá trị $weight$ bất kì. Chúng ta sẽ cho neuron giá trị của $input$, và $expected\_output$ mong muốn tương ứng với $input$ này. Với giá trị $weight$ ban đầu, neuron áp dụng công thức tính ra $output$, và tất nhiên sẽ có sai số với $expected\_output$.

Để giúp neuron điều chỉnh giá trị $weight$ cho phù hợp (sao cho $output$ của neuron sẽ gần hơn với $expected\_output$), chúng ta sẽ cung cấp cho neuron một hàm tính sai số. Sai số càng lớn khi $output$ của neuron càng xa với $expected\_output$ mong muốn, và neuron sẽ thay đổi $weight$ nhiều hơn khi sai số càng lớn.

TODO: Hình vẽ neuron với input, weight, output, expected_output

Hiển nhiên, $weight$ sẽ biến thiên tùy vào $input$ và $expected\_output$ trong quá trình học. Do đó, ta sẽ sử dụng biến TensorFlow ($tf.Variable$ class) cho $weight$. Code của $tf.Variable$ class: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/ops/variables.py.

Đầu tiên, khởi tạo biến này với một giá trị ban đầu là $0.8$.

In [19]:
weight = tf.Variable(0.8)
weight

Hãy xem so với khởi tạo một hằng số, việc khởi tạo một biến thêm vào graph bao nhiêu operations?

In [20]:
for op in graph.get_operations():
    print(op.name)

Const
Variable/initial_value
Variable
Variable/Assign
Variable/read


Vậy là có 4 operations tương ứng với việc có thêm biến này, bên cạnh operation khởi tạo hằng số ở phần 1.
* Variable/initial_value
* Variable
* Variable/Assign
* Variable/read

Với $input\_value$ $1.0$ từ phần 1 và $weight$ vừa khởi tạo, ta đã sẵn sàng thêm vào graph một operation tính toán thật sự khi tính $output$.

In [21]:
output = weight * input_value

Hãy viết code để biết được loại operation nào vừa được thêm vào graph.

In [22]:
#answer:


Bạn có thể đoán được đoạn code có mục đich gì không?

In [23]:
for op_input in op.inputs:
    print(op_input)

The multiplication operation tracks where its inputs come from (other operations in the graph). Imagine as the graph grows in size, it would become really hard to keep track of their relationship without having a tool to visualize the whole graph.  [TensorBoard graph visualization](https://www.tensorflow.org/how_tos/graph_viz/) is designed to help.

How to find out the result of this multiplication? We need the "session" to "run" the "output" operation.

In [13]:
sess.run(output)

FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[Node: Variable/read = Identity[T=DT_FLOAT, _class=["loc:@Variable"], _device="/job:localhost/replica:0/task:0/cpu:0"](Variable)]]

Caused by op 'Variable/read', defined at:
  File "C:\Users\vumin\Anaconda3\envs\tensorflow\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\vumin\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\vumin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\vumin\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\vumin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\vumin\Anaconda3\envs\tensorflow\lib\site-packages\zmq\eventloo

But we got this error instead of the result: "FailedPreconditionError (see above for traceback): Attempting to use uninitialized value Variable_2". It turns out that the operation requires the variable "weight". Even though we had initialized this variable, its value hasn't yet been set in the current session. The tf.global_variables_initializer() function will generate an operation that initializes all our variables (in this case just one, "weight").

In [83]:
init = tf.global_variables_initializer()

Don't forget to run this "init" operation.

In [84]:
sess.run(init)

Now we can run the "output" operation.

In [16]:
sess.run(output)

0.80000001

Wonder why the result of 0.8\*1.0 is not exactly 0.8?

## See your graph in TensorBoard

This tool helps visualize your graph in a nice diagram.
First create a FileWriter with the name of the output directory "log_simple_graph", and of course the graph itself.

In [40]:
file_writer = tf.summary.FileWriter("log_simple_graph", sess.graph)

Now come back to the command line and start up the TensorBoard.

In [41]:
#(C:Anaconda3) D:\MaSSP\track_2>ls
#Tutorial 1 - Overview.ipynb  data  log_simple_graph

#(C:Anaconda3) D:\MaSSP\track_2>activate tensorflow

#(tensorflow) D:\MaSSP\track_2>tensorboard --logdir=log_simple_graph
#WARNING:tensorflow:Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
#WARNING:tensorflow:Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
#Starting TensorBoard b'39' on port 6006
#(You can navigate to http://10.0.0.5:6006)

Note that a directory "log_simple_graph" is created, and a URL is given to show the diagram.

![log_simple_graph](https://github.com/chauvm/tensorflow_tutorials/raw/master/images/log_simple_graph.png "TensorBoard's diagram of our graph")


Note that an operation has its name field that is different from the Python variable name (you won't see "output", "input_value" in the graph, but "Const_1" or "Variable_2" instead). We should name the operations to make them show up better in the graph.

In [42]:
x = tf.constant(1.0, name="input")
w = tf.Variable(0.8, name="weight")
y = tf.mul(w, x, name="output")

In [43]:
file_writer_2 = tf.summary.FileWriter("log_simple_graph_2", sess.graph)

Now restart TensorBoard with "log_simple_graph_2", and refresh the page.

In [ ]:
#(tensorflow) D:\MaSSP\track_2>ls
#Tutorial 1 - Overview.ipynb  data  log_simple_graph  log_simple_graph_2

#(tensorflow) D:\MaSSP\track_2>tensorboard --logdir=log_simple_graph_2
#Starting TensorBoard b'39' on port 6006
#(You can navigate to http://10.0.0.5:6006)

We see the operation names in the graph.
![log_simple_graph_2](https://github.com/chauvm/tensorflow_tutorials/raw/master/images/named_vars.png "TensorBoard's diagram of our graph")